<a href="https://colab.research.google.com/github/shernee/04_cmpe258/blob/master/Data_augmentation_classification_%5Bk%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras
!pip install tensorflow
!pip install nlpaug
!pip install opencv-python-headless
!pip install av
!pip install -U augly[video]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 4.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 44.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [45]:
!pip install tsaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!sudo apt-get install python3-magic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  python3-magic
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 9,376 B of archives.
After this operation, 43.0 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 python3-magic all 2:0.4.15-3 [9,376 B]
Fetched 9,376 B in 1s (14.1 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package python3-magic.
(Reading database ... 122349 files and dire

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import urllib.request
import zipfile
import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Embedding, GlobalMaxPooling1D, LSTM
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

import nlpaug.augmenter.word as naw
import augly.video as vidaugs
import tsaug
from tsaug import TimeWarp, Crop, Drift, AddNoise, Reverse


### Tabular data

In [6]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls

--2023-04-12 18:17:07--  https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5539328 (5.3M) [application/x-httpd-php]
Saving to: ‘default of credit card clients.xls’

default of credit c 100%[===================>]   5.28M  4.48MB/s    in 1.2s    

2023-04-12 18:17:09 (4.48 MB/s) - ‘default of credit card clients.xls’ saved [5539328/5539328]



preprocess the data by splitting it into a training and testing set, normalizing the features, and one-hot encoding the target variable

In [7]:
df = pd.read_excel('default of credit card clients.xls', header=1)

In [8]:
# Separate the features and target variable
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Split the data into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# One-hot encode the target variable
encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test = encoder.transform(y_test.reshape(-1, 1)).toarray()

Before Augmentation

In [9]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
750/750 [==============================] - 4s 4ms/step - loss: 0.4702 - accuracy: 0.8087 - val_loss: 0.4492 - val_accuracy: 0.8183
Epoch 2/10
750/750 [==============================] - 2s 2ms/step - loss: 0.4404 - accuracy: 0.8183 - val_loss: 0.4383 - val_accuracy: 0.8192
Epoch 3/10
750/750 [==============================] - 2s 2ms/step - loss: 0.4338 - accuracy: 0.8196 - val_loss: 0.4383 - val_accuracy: 0.8190
Epoch 4/10
750/750 [==============================] - 2s 2ms/step - loss: 0.4297 - accuracy: 0.8208 - val_loss: 0.4396 - val_accuracy: 0.8177
Epoch 5/10
750/750 [==============================] - 2s 2ms/step - loss: 0.4275 - accuracy: 0.8225 - val_loss: 0.4373 - val_accuracy: 0.8188
Epoch 6/10
750/750 [==============================] - 2s 2ms/step - loss: 0.4252 - accuracy: 0.8228 - val_loss: 0.4404 - val_accuracy: 0.8142
Epoch 7/10
750/750 [==============================] - 2s 2ms/step - loss: 0.4236 - accuracy: 0.8217 - val_loss: 0.4354 - val_accuracy: 0.8168
Epoch 

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)

188/188 [==============================] - 1s 3ms/step - loss: 0.4403 - accuracy: 0.8178
Test accuracy: 0.8178333044052124


One way to add noise to the data is by adding random Gaussian noise to each feature with a given mean and sd

In [11]:
def add_noise(feature, mean, std):
    noise = np.random.normal(mean, std, len(feature))
    return feature + noise

# Add noise to each feature of the dataset
for i in range(1, len(df.columns)):
    df.iloc[:, i] = add_noise(df.iloc[:, i], 0, 0.1)

In [12]:
# Separate features and labels
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split the data into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:

# Define the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=2)

# Evaluate the model on the testing set
score = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", score[1])

Epoch 1/50
750/750 - 2s - loss: 0.4874 - accuracy: 0.0000e+00 - 2s/epoch - 3ms/step
Epoch 2/50
750/750 - 1s - loss: 0.4542 - accuracy: 0.0000e+00 - 1s/epoch - 2ms/step
Epoch 3/50
750/750 - 1s - loss: 0.4452 - accuracy: 0.0000e+00 - 1s/epoch - 1ms/step
Epoch 4/50
750/750 - 1s - loss: 0.4408 - accuracy: 0.0000e+00 - 1s/epoch - 1ms/step
Epoch 5/50
750/750 - 1s - loss: 0.4393 - accuracy: 0.0000e+00 - 1s/epoch - 2ms/step
Epoch 6/50
750/750 - 2s - loss: 0.4358 - accuracy: 0.0000e+00 - 2s/epoch - 2ms/step
Epoch 7/50
750/750 - 1s - loss: 0.4344 - accuracy: 0.0000e+00 - 1s/epoch - 2ms/step
Epoch 8/50
750/750 - 1s - loss: 0.4324 - accuracy: 0.0000e+00 - 1s/epoch - 1ms/step
Epoch 9/50
750/750 - 1s - loss: 0.4311 - accuracy: 0.0000e+00 - 1s/epoch - 2ms/step
Epoch 10/50
750/750 - 1s - loss: 0.4317 - accuracy: 0.0000e+00 - 1s/epoch - 2ms/step
Epoch 11/50
750/750 - 1s - loss: 0.4297 - accuracy: 0.0000e+00 - 1s/epoch - 2ms/step
Epoch 12/50
750/750 - 1s - loss: 0.4293 - accuracy: 0.0000e+00 - 1s/epoch 

Added random Gaussian noise with mean 0 and standard deviation 0.1 to each feature of the dataset, except for the target variable.

Adding noise to the data can help in making the model more robust to noise in the input data, but it may not always improve the performance of the model.

It reduces loss but worsens test accuracy

### Image

In [14]:
# Download dataset
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
filename = 'cats_and_dogs_filtered.zip'
urllib.request.urlretrieve(url, filename)

# Extract dataset
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('.')

In [15]:
# Set up directories
base_dir = 'cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'validation')

# Set up data generators with data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [16]:
# Define and compile model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])


In [17]:
# Train model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=50,
    epochs=5,
    validation_data=test_generator,
    validation_steps=50)

# Evaluate model
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('Test accuracy:', test_acc)

<ipython-input-17-49310b23b6bb>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/5
50/50 [==============================] - 81s 2s/step - loss: 0.6973 - accuracy: 0.5080 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 2/5
50/50 [==============================] - 81s 2s/step - loss: 0.6938 - accuracy: 0.5060 - val_loss: 0.7697 - val_accuracy: 0.5000
Epoch 3/5
50/50 [==============================] - 84s 2s/step - loss: 0.6927 - accuracy: 0.5220 - val_loss: 0.6873 - val_accuracy: 0.6110
Epoch 4/5
50/50 [==============================] - 81s 2s/step - loss: 0.6949 - accuracy: 0.5190 - val_loss: 0.6877 - val_accuracy: 0.5500
Epoch 5/5
50/50 [==============================] - 78s 2s/step - loss: 0.6992 - accuracy: 0.5240 - val_loss: 0.6722 - val_accuracy: 0.6350


<ipython-input-17-49310b23b6bb>:10: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)


Test accuracy: 0.6349999904632568


### Text

In [18]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 [==============================] - 1s 0us/step


In [22]:
maxlen = 200
train_data = pad_sequences(train_data, maxlen=maxlen)
test_data = pad_sequences(test_data, maxlen=maxlen)

x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [23]:
# define the model using an embedding layer, a global max pooling layer, and two dense layers with dropout regularization

embedding_dim = 16

model = Sequential([
    Embedding(10000, embedding_dim, input_length=maxlen),
    GlobalMaxPooling1D(),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=40,
    batch_size=512,
    validation_data=(x_val, y_val)
)
# During training, Keras will automatically apply data augmentation techniques like shuffling and random cropping to the input data to prevent overfitting


Epoch 1/40
30/30 [==============================] - 2s 30ms/step - loss: 0.6927 - accuracy: 0.5178 - val_loss: 0.6914 - val_accuracy: 0.5571
Epoch 2/40
30/30 [==============================] - 1s 27ms/step - loss: 0.6892 - accuracy: 0.5597 - val_loss: 0.6862 - val_accuracy: 0.6409
Epoch 3/40
30/30 [==============================] - 1s 27ms/step - loss: 0.6804 - accuracy: 0.6211 - val_loss: 0.6745 - val_accuracy: 0.6476
Epoch 4/40
30/30 [==============================] - 1s 37ms/step - loss: 0.6627 - accuracy: 0.6751 - val_loss: 0.6516 - val_accuracy: 0.7605
Epoch 5/40
30/30 [==============================] - 1s 41ms/step - loss: 0.6322 - accuracy: 0.7307 - val_loss: 0.6139 - val_accuracy: 0.7990
Epoch 6/40
30/30 [==============================] - 1s 38ms/step - loss: 0.5837 - accuracy: 0.7725 - val_loss: 0.5614 - val_accuracy: 0.8106
Epoch 7/40
30/30 [==============================] - 1s 27ms/step - loss: 0.5262 - accuracy: 0.7969 - val_loss: 0.5014 - val_accuracy: 0.8183
Epoch 8/40
30

In [25]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 1s 2ms/step - loss: 0.6128 - accuracy: 0.8078
[0.6128406524658203, 0.8077600002288818]


In [26]:

# Define the augmentation function
def augment_text(text, aug):
    augmented_text = aug.augment(text)
    return augmented_text

# Define the augmentation method
aug = naw.SynonymAug(aug_src='wordnet')


# Augment the text data
augmented_x_train = [aug.augment(str(text)) for text in partial_x_train]
augmented_x_val = [aug.augment(str(text)) for text in x_val]
augmented_test_data = [aug.augment(str(text)) for text in test_data]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [27]:
# Create a tokenizer object
tokenizer = Tokenizer(num_words=10000)
augmented_x_train = tokenizer.texts_to_sequences(augmented_x_train)
augmented_x_val = tokenizer.texts_to_sequences(augmented_x_val)
augmented_test_data = tokenizer.texts_to_sequences(augmented_test_data)

# Pad the augmented data
augmented_x_train = pad_sequences(augmented_x_train, maxlen=maxlen)
augmented_x_val = pad_sequences(augmented_x_val, maxlen=maxlen)
augmented_test_data = pad_sequences(augmented_test_data, maxlen=maxlen)

In [28]:
# Train the model with augmented data
history = model.fit(
    augmented_x_train,
    partial_y_train,
    epochs=40,
    batch_size=512,
    validation_data=(augmented_x_val, y_val)
)

# Evaluate the model with the augmented test data
results = model.evaluate(augmented_test_data, test_labels)
print(results)

Epoch 1/40
30/30 [==============================] - 1s 24ms/step - loss: 0.7361 - accuracy: 0.4971 - val_loss: 0.6964 - val_accuracy: 0.4947
Epoch 2/40
30/30 [==============================] - 1s 21ms/step - loss: 0.6954 - accuracy: 0.5068 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 3/40
30/30 [==============================] - 1s 24ms/step - loss: 0.6938 - accuracy: 0.5008 - val_loss: 0.6940 - val_accuracy: 0.4947
Epoch 4/40
30/30 [==============================] - 1s 24ms/step - loss: 0.6935 - accuracy: 0.4963 - val_loss: 0.6933 - val_accuracy: 0.4947
Epoch 5/40
30/30 [==============================] - 1s 21ms/step - loss: 0.6932 - accuracy: 0.5040 - val_loss: 0.6932 - val_accuracy: 0.4947
Epoch 6/40
30/30 [==============================] - 1s 30ms/step - loss: 0.6933 - accuracy: 0.4975 - val_loss: 0.6932 - val_accuracy: 0.4947
Epoch 7/40
30/30 [==============================] - 1s 31ms/step - loss: 0.6932 - accuracy: 0.5030 - val_loss: 0.6934 - val_accuracy: 0.4947
Epoch 8/40
30

### Video

In [32]:
!wget -c http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
!unrar x hmdb51_org.rar
!unrar x shoot_gun.rar
!unrar x brush_hair.rar

--2023-04-12 19:23:01--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar [following]
--2023-04-12 19:23:01--  https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from hmdb51_org.rar


Would you like to replace the existing file shoot_gun.rar
32372358 bytes, modified on 2011-08-25 12:21
with a new one
32372358 by

In [35]:
data_dir_1 = './brush_hair'
data_dir_2 = './shoot_gun'
img_size = 64

# Read frames from videos in the first directory
data_1 = []
for i, filename in enumerate(os.listdir(data_dir_1)):
    if i == 5:
        break
    video_path = os.path.join(data_dir_1, filename)
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (img_size, img_size))
        data_1.append(frame)
    cap.release()

# Read frames from videos in the second directory
data_2 = []
for i, filename in enumerate(os.listdir(data_dir_2)):
    if i == 5:
        break
    video_path = os.path.join(data_dir_2, filename)
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (img_size, img_size))
        data_2.append(frame)
    cap.release()

# Create labels for the data
labels_1 = np.zeros(len(data_1))
labels_2 = np.ones(len(data_2))

# Combine the data and labels
data = np.array(data_1 + data_2)
labels = np.concatenate((labels_1, labels_2))

# Shuffle the data and labels
idx = np.random.permutation(len(data))
data = data[idx]
labels = labels[idx]

# Split the data into training and testing sets
split = int(0.8 * len(data))
x_train = data[:split]
y_train = labels[:split]
x_test = data[split:]
y_test = labels[split:]


In [36]:

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
47/47 [==============================] - 13s 247ms/step - loss: 4.0656 - accuracy: 0.8879 - val_loss: 0.0130 - val_accuracy: 1.0000
Epoch 2/10
47/47 [==============================] - 10s 208ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 9.8609e-04 - val_accuracy: 1.0000
Epoch 3/10
47/47 [==============================] - 11s 244ms/step - loss: 6.7742e-04 - accuracy: 1.0000 - val_loss: 4.0817e-04 - val_accuracy: 1.0000
Epoch 4/10
47/47 [==============================] - 11s 241ms/step - loss: 2.2440e-04 - accuracy: 1.0000 - val_loss: 2.8573e-04 - val_accuracy: 1.0000
Epoch 5/10
47/47 [==============================] - 11s 243ms/step - loss: 1.0139e-04 - accuracy: 1.0000 - val_loss: 7.2348e-05 - val_accuracy: 1.0000
Epoch 6/10
47/47 [==============================] - 11s 244ms/step - loss: 8.2795e-05 - accuracy: 1.0000 - val_loss: 7.8565e-05 - val_accuracy: 1.0000
Epoch 7/10
47/47 [==============================] - 10s 222ms/step - loss: 5.5211e-05 - accuracy: 1.0000 -

In [37]:
# Define the augmentations to be applied to the videos
aug = vidaugs.Compose([
    vidaugs.AddNoise(),
    vidaugs.Blur(sigma=5.0),
    vidaugs.OverlayDots(),
])

# Define the paths to the video files
videos_path = "./"
shoot_gun_path = os.path.join(videos_path, "shoot_gun")
brush_hair_path = os.path.join(videos_path, "brush_hair")

# Define the size of the video frames
img_size = 224

# Define the data lists
data_1 = []
labels_1 = []
data_2 = []
labels_2 = []


In [38]:
# Read the first 5 videos from the shoot_gun folder
for i, filename in enumerate(os.listdir(shoot_gun_path)[:5]):
    if filename.endswith(".avi"):
        filepath = os.path.join(shoot_gun_path, filename)
        cap = cv2.VideoCapture(filepath)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (img_size, img_size))
            writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (img_size, img_size), True)
            writer.write(frame)
            writer.release()
            aug_video = cv2.VideoCapture("output.avi")
            aug_frames = []
            while aug_video.isOpened():
                ret, frame = aug_video.read()
                if not ret:
                    break
                aug_frames.append(frame)
            aug_video.release()
            os.remove("output.avi")
            aug_data = np.array(aug_frames)
            data_1.append(aug_data)
        cap.release()

# Read the first 5 videos from the brush_hair folder
for i, filename in enumerate(os.listdir(brush_hair_path)[:5]):
    if filename.endswith(".avi"):
        filepath = os.path.join(brush_hair_path, filename)
        cap = cv2.VideoCapture(filepath)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (img_size, img_size))
            writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (img_size, img_size), True)
            writer.write(frame)
            writer.release()
            aug_video = cv2.VideoCapture("output.avi")
            aug_frames = []
            while aug_video.isOpened():
                ret, frame = aug_video.read()
                if not ret:
                    break
                aug_frames.append(frame)
            aug_video.release()
            os.remove("output.avi")
            aug_data = np.array(aug_frames)
            data_2.append(aug_data)
        cap.release()

print(len(data_1))
print(len(data_2))

382
1492


In [39]:
# Convert the data lists to numpy arrays
data_1 = np.concatenate(data_1, axis=0)
data_2 = np.concatenate(data_2, axis=0)


# Create the labels for the data
labels_1 = np.zeros((data_1.shape[0],))
labels_2 = np.ones((data_2.shape[0],))

# Merge the data and labels
X = np.concatenate((data_1, data_2))
y = np.concatenate((labels_1, labels_2))

# Shuffle the data and labels
p = np.random.permutation(X.shape[0])
X = X[p]
y = y[p]

# Split the data and labels into training and testing sets
split = int(0.8 * X.shape[0])
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [40]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1499, 224, 224, 3)
(375, 224, 224, 3)
(1499,)
(375,)


In [41]:
# Define the CNN model architecture
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
47/47 [==============================] - 204s 4s/step - loss: 5.0199 - accuracy: 0.9320 - val_loss: 1.2889e-05 - val_accuracy: 1.0000
Epoch 2/10
47/47 [==============================] - 197s 4s/step - loss: 4.4040e-04 - accuracy: 1.0000 - val_loss: 1.8326e-05 - val_accuracy: 1.0000
Epoch 3/10
47/47 [==============================] - 198s 4s/step - loss: 0.0848 - accuracy: 0.9813 - val_loss: 0.0044 - val_accuracy: 1.0000
Epoch 4/10
47/47 [==============================] - 188s 4s/step - loss: 0.0305 - accuracy: 0.9920 - val_loss: 0.0037 - val_accuracy: 0.9973
Epoch 5/10
47/47 [==============================] - 203s 4s/step - loss: 0.0060 - accuracy: 0.9967 - val_loss: 0.0019 - val_accuracy: 0.9973
Epoch 6/10
47/47 [==============================] - 197s 4s/step - loss: 0.0047 - accuracy: 0.9993 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 7/10
47/47 [==============================] - 213s 5s/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
E

In [42]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

12/12 [==============================] - 13s 989ms/step - loss: 0.0011 - accuracy: 1.0000
Test accuracy: 1.0


### Timeseries

In [43]:
# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
dataset = pd.read_csv(url, header=0, index_col=0)

# Convert the dataset to a numpy array
data = dataset.values

# Split the dataset into training and testing sets
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

# Function to create the time series dataset
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Reshape the data into the appropriate format for the neural network
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

# Define the neural network model
model = keras.Sequential([
    keras.layers.LSTM(4, input_shape=(look_back, 1)),
    keras.layers.Dense(1)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)

# Evaluate the model on the testing set
scores = model.evaluate(testX, testY, verbose=0)
print("Test loss:", scores)


Epoch 1/20
2441/2441 - 7s - loss: 57.4063 - 7s/epoch - 3ms/step
Epoch 2/20
2441/2441 - 5s - loss: 15.7742 - 5s/epoch - 2ms/step
Epoch 3/20
2441/2441 - 5s - loss: 9.4864 - 5s/epoch - 2ms/step
Epoch 4/20
2441/2441 - 6s - loss: 7.8869 - 6s/epoch - 2ms/step
Epoch 5/20
2441/2441 - 5s - loss: 7.3149 - 5s/epoch - 2ms/step
Epoch 6/20
2441/2441 - 6s - loss: 7.0568 - 6s/epoch - 2ms/step
Epoch 7/20
2441/2441 - 5s - loss: 6.8638 - 5s/epoch - 2ms/step
Epoch 8/20
2441/2441 - 5s - loss: 6.8072 - 5s/epoch - 2ms/step
Epoch 9/20
2441/2441 - 6s - loss: 6.7023 - 6s/epoch - 2ms/step
Epoch 10/20
2441/2441 - 5s - loss: 6.6606 - 5s/epoch - 2ms/step
Epoch 11/20
2441/2441 - 5s - loss: 6.6241 - 5s/epoch - 2ms/step
Epoch 12/20
2441/2441 - 5s - loss: 6.5955 - 5s/epoch - 2ms/step
Epoch 13/20
2441/2441 - 5s - loss: 6.5841 - 5s/epoch - 2ms/step
Epoch 14/20
2441/2441 - 6s - loss: 6.5591 - 6s/epoch - 2ms/step
Epoch 15/20
2441/2441 - 5s - loss: 6.5565 - 5s/epoch - 2ms/step
Epoch 16/20
2441/2441 - 5s - loss: 6.5357 - 5s/

In [47]:
# Define the augmentations
augmenter = (
    TimeWarp() 
    + Drift(max_drift=(0.1, 0.3)) 
    + Reverse()
)

# Apply the augmentations to the training data
trainX_aug = np.zeros_like(trainX)
for i in range(len(trainX)):
    trainX_aug[i, :, 0] = augmenter.augment(trainX[i, :, 0])
    
# Apply the augmentations to the testing data
testX_aug = np.zeros_like(testX)
for i in range(len(testX)):
    testX_aug[i, :, 0] = augmenter.augment(testX[i, :, 0])

In [48]:
# Train the model on the augmented training data
model.fit(trainX_aug, trainY, epochs=20, batch_size=1, verbose=2)

# Evaluate the model on the augmented testing data
scores_aug = model.evaluate(testX_aug, testY, verbose=0)
print("Test loss (augmented data):", scores_aug)

Epoch 1/20
2441/2441 - 9s - loss: 8.8029 - 9s/epoch - 4ms/step
Epoch 2/20
2441/2441 - 5s - loss: 7.8154 - 5s/epoch - 2ms/step
Epoch 3/20
2441/2441 - 6s - loss: 7.4775 - 6s/epoch - 2ms/step
Epoch 4/20
2441/2441 - 5s - loss: 7.2722 - 5s/epoch - 2ms/step
Epoch 5/20
2441/2441 - 6s - loss: 7.1028 - 6s/epoch - 2ms/step
Epoch 6/20
2441/2441 - 5s - loss: 7.0296 - 5s/epoch - 2ms/step
Epoch 7/20
2441/2441 - 5s - loss: 6.9428 - 5s/epoch - 2ms/step
Epoch 8/20
2441/2441 - 6s - loss: 6.9600 - 6s/epoch - 2ms/step
Epoch 9/20
2441/2441 - 5s - loss: 6.9180 - 5s/epoch - 2ms/step
Epoch 10/20
2441/2441 - 5s - loss: 6.8955 - 5s/epoch - 2ms/step
Epoch 11/20
2441/2441 - 5s - loss: 6.8526 - 5s/epoch - 2ms/step
Epoch 12/20
2441/2441 - 5s - loss: 6.8624 - 5s/epoch - 2ms/step
Epoch 13/20
2441/2441 - 6s - loss: 6.8626 - 6s/epoch - 2ms/step
Epoch 14/20
2441/2441 - 5s - loss: 6.8595 - 5s/epoch - 2ms/step
Epoch 15/20
2441/2441 - 5s - loss: 6.8405 - 5s/epoch - 2ms/step
Epoch 16/20
2441/2441 - 5s - loss: 6.8195 - 5s/ep